In [3]:
# importing libraries
import cv2
import numpy as np
import math

In [85]:
def mid_line_equation(x_inter, y_inter, x_l, y_l, x_r, y_r):
    x_mid = (x_l + x_r)/2
    y_mid = (y_l + y_r)/2
    a_mid = (y_mid - y_inter)/(x_mid-x_inter+0.00001)
    b_mid = y_mid - a_mid * x_mid

    theta = np.arctan(-1/a_mid)
    rho = np.sin(theta) * b_mid

    print('Mid line equation:')

    print('y = {a}x + {b}'.format(a = a_mid, b = b_mid))
    print('theta = {th}, rho = {r}\n'.format(th = theta, r = rho))
    return (int(y_inter), int(x_inter), int(y_mid), int(x_mid), int(x_inter), int(y_inter), a_mid, b_mid)


def detect_mid_line(ptl1, ptl2, ptr1, ptr2):
    #calculate the center line
    y_l1 = ptl1[0]
    x_l1 = ptl1[1]
    y_l2 = ptl2[0]
    x_l2 = ptl2[1]

    a_l = (y_l1-y_l2)/(x_l1-x_l2+0.000001)
    b_l = y_l1 - x_l1 * a_l

    y_r1 = ptr1[0]
    x_r1 = ptr1[1]
    y_r2 = ptr2[0]
    x_r2 = ptr2[1]

    a_r = (y_r1-y_r2)/(x_r1-x_r2+0.0000001)
    b_r = y_r1 - x_r1 * a_r

    #get intersect point
    x_inter = (b_r-b_l)/(a_l-a_r+0.00001)
    y_inter = x_inter*a_l + b_l

    #get mid_point left
    x_ml = x_inter + 10000
    y_ml = x_ml * a_l + b_l
    d_2 = np.square(y_inter - y_ml) + np.square(x_inter - x_ml)

    #get mid_point right
    #delta = b^2 - 4ac
    a_mr = np.square(a_r) + 1
    b_mr = -2*(x_inter+y_inter*a_r-a_r*b_r)
    c_mr = -(d_2 - np.square(x_inter) - np.square(y_inter) + 2*y_inter*b_r - np.square(b_r))
    delta = np.square(b_mr) - 4*a_mr*c_mr

    #get x_mid_right and y_mid_right
    x_mr1 = (-b_mr - np.sqrt(delta)) / (2*a_mr+0.000001)
    x_mr2 = (-b_mr + np.sqrt(delta)) / (2*a_mr+0.000001)

    y_mr1 = a_r * x_mr1 + b_r
    y_mr2 = a_r * x_mr2 + b_r

    #return mid_line
    if x_mr1 > x_inter:
        return mid_line_equation(x_inter, y_inter, x_ml, y_ml, x_mr1, y_mr1)
    else:
        return mid_line_equation(x_inter, y_inter, x_ml, y_ml, x_mr2, y_mr2)

In [116]:
def lane_making(img):
    #import image
    #cover to hsv
    blur_img = cv2.GaussianBlur(img,(5, 5), 2)
    img_gray = cv2.cvtColor(blur_img, cv2.COLOR_BGR2GRAY)

    #blur
    line_mask = np.ones((len(img), len(img[0])))
    line_mask[:int(len(img)/2),:] = 0
    line_mask[-int(len(img)/15):, :] = 0

    line_img = line_mask*img_gray
    blur_img = cv2.GaussianBlur(line_img,(5, 5), 5)

    kernel1 = np.ones((5, 5), np.uint8)
    erosion = cv2.erode(blur_img, kernel1, iterations=8)
    dilation = cv2.dilate(erosion, kernel1, iterations=1)

    bw = cv2.threshold(dilation,210,255,cv2.THRESH_BINARY_INV)[1]

    #find edges
    t_lower = 0  # Lower Threshold
    t_upper = 255 # Upper threshold

    # Applying the Canny Edge filter
    edges = cv2.Canny(np.uint8(bw), t_lower, t_upper)
    lines = cv2.HoughLines(edges, 1, np.pi / 50, 50, None, 0, 0)

    if lines is None:
        cv2.putText(img, 'None', (50,100), cv2.FONT_HERSHEY_SIMPLEX, 1, (100,0,255), 2)
        cv2.imshow('Frame', img)
        return None, None, None, None
    else:
        len_line = len(lines)
        get_min = False
        get_max = False

        if len(lines) < 3:
            cv2.putText(img, 'None', (50,100), cv2.FONT_HERSHEY_SIMPLEX, 1, (100,0,255), 2)
            cv2.imshow('Frame', img)
            return None, None, None, None
        else:
            #draw lines
            min_pt = [len(img[0]), 0, 0]
            max_pt = [0, 0, 0]

            #get 2 main lines
            for i in range(0, len(lines)):
                rho = lines[i][0][0]
                theta = lines[i][0][1]
                a = math.cos(theta)
                b = math.sin(theta)
                x0 = a * rho
                y0 = b * rho
                pt1 = (int(x0 + 1000*(-b)), int(y0 + 1000*(a)))
                pt2 = (int(x0 - 1000*(-b)), int(y0 - 1000*(a)))

                if abs(pt1[1]-pt2[1]) >30:
                    if min_pt[0] > rho:
                        min_pt[0] = x0
                        min_pt[1] = pt1
                        min_pt[2] = pt2
                        get_min = True


                    if max_pt[0] < rho:
                        max_pt[0] = x0
                        max_pt[1] = pt1
                        max_pt[2] = pt2
                        get_max = True
                else:
                    len_line -= 1

            #get center line

            if len_line > 2 and get_min and get_max:
                #get center

                center = detect_mid_line(min_pt[1], min_pt[2], max_pt[1], max_pt[2])
                x_inter = center[4]
                y_inter = center[5]
                a_mid = center[6]
                b_mid = center[7]

                if x_inter < 0.5 * len(img):
                    cv2.putText(img, ' y = {a}x + {b}'.format(a = round(a_mid,3), b = round(b_mid,3)), (50,100), cv2.FONT_HERSHEY_SIMPLEX, 1, (100,0,255), 2)
                    cv2.line(img, min_pt[1], min_pt[2], (100,0,255), 3, cv2.LINE_AA)
                    cv2.line(img, max_pt[1], max_pt[2], (100,0,255), 3, cv2.LINE_AA)
                    cv2.line(img, (center[0], center[1]), (center[2], center[3]), (255,100,0), 3, cv2.LINE_AA)
                    cv2.imshow('Frame', img)
                    return x_inter, y_inter, a_mid, b_mid
                else:
                    cv2.putText(img, 'None', (50,100), cv2.FONT_HERSHEY_SIMPLEX, 1, (100,0,255), 2)
                    cv2.imshow('Frame', img)
                    return None, None, None, None
            else:
                cv2.putText(img, 'None', (50,100), cv2.FONT_HERSHEY_SIMPLEX, 1, (100,0,255), 2)
                cv2.imshow('Frame', img)
                return None, None, None, None


In [118]:
# Create a VideoCapture object and read from input fileqqqqq
cap = cv2.VideoCapture("./dataset/videos/OneLine/oneL_1.mp4")

# Check if camera opened successfully
if (cap.isOpened()== False):
    print("Error opening video file")

# Read until video is completed
while(cap.isOpened()):
    # Capture frame-by-frame
    ret, frame = cap.read()
    if ret == True:
        x_inter, y_inter, a_mid, b_mid = lane_making(frame)

    # Press Q on keyboard to exit
        if cv2.waitKey(25) & 0xFF == ord('q'):
            break

# Break the loop
    else:
        break

# When everything done, release
# the video capture object
cap.release()

# Closes all the frames
cv2.destroyAllWindows()

Mid line equation:
y = -0.15846416269144656x + 747.4198520846883
theta = 1.413638923651796, rho = 738.208778105314

Mid line equation:
y = -0.1584641632201018x + 754.2493418396236
theta = 1.4136389231360906, rho = 744.9541023397587

Mid line equation:
y = -0.03154729731308782x + 721.4527919621385
theta = 1.5392594888630733, rho = 721.094053270207

Mid line equation:
y = -0.15846416272881017x + 758.4078009846746
theta = 1.4136389236153477, rho = 749.0613134045634

Mid line equation:
y = -0.12628529962916574x + 746.066020024294
theta = 1.4451760072974766, rho = 740.1871228699636

Mid line equation:
y = -0.15846416023498835x + 765.2121477948297
theta = 1.4136389260480815, rho = 755.7818049037584

Mid line equation:
y = -0.15846416325746537x + 766.23729073953
theta = 1.4136389230996422, rho = 756.7943138095443

Mid line equation:
y = -0.1584641632574654x + 765.2372907395297
theta = 1.4136389230996422, rho = 755.806637638003

Mid line equation:
y = -0.15846416023498835x + 764.2121477948297


C:\Users\Asus\AppData\Local\Temp\ipykernel_27024\2961138534.py:7: RuntimeWarning: divide by zero encountered in double_scalars
  theta = np.arctan(-1/a_mid)


Mid line equation:
y = -0.22348072417086645x + 1049.4329233216565
theta = 1.3509284226979916, rho = 1024.169156431655

Mid line equation:
y = -0.03129736627615515x + 994.2207136431352
theta = 1.5395091733694841, rho = 993.7341389832684

Mid line equation:
y = -0.32525831534553173x + 1018.2848448183067
theta = 1.2563308080584328, rho = 968.3500237645167

Mid line equation:
y = -0.32519205735552836x + 1051.2416152965088
theta = 1.2563907282089097, rho = 999.7101363712492

Mid line equation:
y = 0.00015000116573347683x + 937.999650281457
theta = -1.5706463256302883, rho = -937.9996397287971

Mid line equation:
y = 0.00020000205782014376x + 901.9991768739426
theta = -1.5705963247397432, rho = -901.9991588335884

Mid line equation:
y = -0.15846425905800274x + 1022.9032236495409
theta = 1.4136388296458153, rho = 1010.2971248295173

Mid line equation:
y = -0.1907077338598863x + 1032.2102849072403
theta = 1.3823513941708068, rho = 1013.9367936753963

Mid line equation:
y = -0.15846426002519745